In [1]:
import sys
import cv2
import numpy as np
from pydensecrf import densecrf as dcrf
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian

In [7]:
print("IMAGE and ANNO are inputs and OUTPUT is where the result should be written.")
print("If there's at least one single full-black pixel in ANNO, black is assumed to mean unknown.")
    
fn_im = './stone6.png'
fn_anno = './depthq.png'
fn_output = './output.png'

img = cv2.imread(fn_im)

# Convert the annotation's RGB color to a single 32-bit integer color 0xBBGGRR
anno_rgb = cv2.imread(fn_anno)
img = cv2.resize(img, (640, 480))
anno_rgb = cv2.resize(anno_rgb, (640, 480)).astype(np.uint32)
anno_lbl = anno_rgb[:,:,0] + (anno_rgb[:,:,1] << 8) + (anno_rgb[:,:,2] << 16)

img = cv2.resize(img, (anno_rgb.shape[1], anno_rgb.shape[0]))
# Convert the 32bit integer color to 1, 2, ... labels.
# Note that all-black, i.e. the value 0 for background will stay 0.
colors, labels = np.unique(anno_lbl, return_inverse=True)
print(colors, labels)
# But remove the all-0 black, that won't exist in the MAP!
HAS_UNKNOWN_LABEL = 0 in colors
if HAS_UNKNOWN_LABEL:
    print("Found a full-black pixel in annotation image, assuming it means 'unknown' label, and will thus not be present in the output!")
    print("If 0 is an actual label for you, consider writing your own code, or simply giving your labels only non-zero values.")
    colors = colors[1:]
#else:
#    print("No single full-black pixel found in annotation image. Assuming there's no 'unknown' label!")

# And create a mapping back from the labels to 32bit integer colors.
colorize = np.empty((len(colors), 3), np.uint8)
colorize[:,0] = (colors & 0x0000FF)
colorize[:,1] = (colors & 0x00FF00) >> 8
colorize[:,2] = (colors & 0xFF0000) >> 16
print(colorize.shape)

# Compute the number of classes in the label image.
# We subtract one because the number shouldn't include the value 0 which stands
# for "unknown" or "unsure".
n_labels = len(set(labels.flat)) - int(HAS_UNKNOWN_LABEL)
print(n_labels, " labels", (" plus \"unknown\" 0: " if HAS_UNKNOWN_LABEL else ""), set(labels.flat))

###########################
### Setup the CRF model ###
###########################

d = dcrf.DenseCRF2D(img.shape[1], img.shape[0], n_labels)

# get unary potentials (neg log probability)
U = unary_from_labels(labels, n_labels, gt_prob=0.9, zero_unsure=HAS_UNKNOWN_LABEL)
d.setUnaryEnergy(U)

# # This adds the color-independent term, features are the locations only.
# d.addPairwiseGaussian(sxy=(3, 3), compat=3, kernel=dcrf.DIAG_KERNEL,
#                       normalization=dcrf.NORMALIZE_SYMMETRIC)

# This adds the color-dependent term, i.e. features are (x,y,r,g,b).
d.addPairwiseBilateral(sxy=(11, 11), srgb=(12, 12, 12), rgbim=img,
                       compat=np.array(['a', 'a']),
                       kernel=dcrf.DIAG_KERNEL,
                       normalization=dcrf.NORMALIZE_SYMMETRIC)

####################################
### Do inference and compute MAP ###
####################################

# Run five inference steps.
Q = d.inference(15)

# Find out the most probable class for each pixel.
MAP = np.argmax(Q, axis=0)

# Convert the MAP (labels) back to the corresponding colors and save the image.
# Note that there is no "unknown" here anymore, no matter what we had at first.
MAP = colorize[MAP,:]
cv2.imwrite(fn_output, MAP.reshape(img.shape))

# Just randomly manually run inference iterations
# Q, tmp1, tmp2 = d.startInference()
# for i in range(15):
#     print("KL-divergence at {}: {}".format(i, d.klDivergence(Q)))
#     d.stepInference(Q, tmp1, tmp2)

IMAGE and ANNO are inputs and OUTPUT is where the result should be written.
If there's at least one single full-black pixel in ANNO, black is assumed to mean unknown.
[       0    65793   131586   197379   263172   394758   460551   526344
   592137   657930   723723   789516   855309   921102   986895  1052688
  1118481  1250067  1315860  1381653  1447446  1513239  1579032  1644825
  1710618  1776411  1842204  1907997  1973790  2039583  2105376  2171169
  2236962  2302755  2368548  2434341  2500134  2565927  2631720  2697513
  2763306  2829099  2894892  2960685  3026478  3158064  3223857  3355443
  3421236  3487029  3552822  3684408  3750201  3815994  3881787  3947580
  4013373  4079166  4144959  4210752  4276545  4342338  4408131  4473924
  4539717  4605510  4671303  4737096  4802889  4868682  4934475  5000268
  5066061  5131854  5197647  5263440  5329233  5395026  5460819  5526612
  5592405  5658198  5723991  5855577  5921370  5987163  6052956  6118749
  6184542  6250335  6316128  6

TypeError: must be real number, not numpy.str_